# 03 — Interactive Decision Helper (single source of truth = seller_windows.csv)

Flow:
1) Auto-find project root that has BOTH `reports/` and `data/processed/`.
2) Load:
   - `data/processed/surplus_kW_10min.csv` (for sizing counterparties)
   - `reports/seller_windows.csv` (authoritative “can sell now?”)
   - `reports/trade_report.csv` (optional: honors cleared trades first)
3) Prompt: home → day (limited to existing days) → AM/PM time (10-min snap).
4) Output: decision string (SELL / BUY / use own/battery / grid), with context.


In [8]:
# 03 — Interactive Decision Helper (full replacement)

from pathlib import Path
import pandas as pd
import numpy as np
import re
import warnings

warnings.filterwarnings("ignore", category=UserWarning)

# --- robust root detection: must have BOTH reports/ and data/processed/ ---
CWD = Path.cwd()
candidates = [CWD, CWD.parent, CWD.parent.parent]
ROOT = None
for c in candidates:
    if (c / "reports").exists() and (c / "data" / "processed").exists():
        ROOT = c
        break
if ROOT is None:
    raise FileNotFoundError("Could not find project root (needs both /reports and /data/processed). "
                            "Run 01 & 02 first; then run 03 from project or notebooks folder.")

PROCESSED = ROOT / "data" / "processed"
REPORTS   = ROOT / "reports"

# --- load authoritative artifacts ---
sup_path = PROCESSED / "surplus_kW_10min.csv"
if not sup_path.exists():
    raise FileNotFoundError(f"Missing: {sup_path.as_posix()}  (Run notebook 01.)")

surplus = pd.read_csv(sup_path, parse_dates=[0], index_col=0).sort_index()

sw_path = REPORTS / "seller_windows.csv"
if not sw_path.exists():
    raise FileNotFoundError(f"Missing: {sw_path.as_posix()}  (Run notebook 02.)")

sw = pd.read_csv(sw_path, parse_dates=["start","end"])
# normalize day col to date for quick matching
sw["day"] = pd.to_datetime(sw["day"]).dt.date

# optional, honored first if exists
trades_path = REPORTS / "trade_report.csv"
report = (pd.read_csv(trades_path, parse_dates=["time"])
          if trades_path.exists()
          else pd.DataFrame(columns=["time","buyer","seller","quantity_kWh"]))

# derive allowed day numbers from surplus index
available_days = pd.DatetimeIndex(surplus.index).normalize().unique()
allowed_day_nums = [d.day for d in available_days]

# small debug (comment out if noisy)
print("ROOT =", ROOT)
print("Loaded:", sup_path.name, "|", sw_path.name, "|",
      (trades_path.name if trades_path.exists() else "no trade_report.csv"))
print("Homes:", ", ".join(surplus.columns))
print("Allowed day numbers:", allowed_day_nums)


ROOT = e:\VPP
Loaded: surplus_kW_10min.csv | seller_windows.csv | trade_report.csv
Homes: h1, h2, h3, h4, h5
Allowed day numbers: [23, 24, 25, 26]


In [9]:
def parse_home(s: str) -> str:
    """Accepts h1..hN; must exist in surplus columns."""
    s = s.strip().lower()
    if re.fullmatch(r"h[1-9]\d*", s) and s in surplus.columns:
        return s
    raise ValueError(f"Home must be one of: {', '.join(surplus.columns)}")

def get_valid_day() -> pd.Timestamp:
    """Prompt user until they pick a day number that exists in the data."""
    opts = ", ".join(str(d) for d in allowed_day_nums)
    while True:
        raw = input(f"Enter the day number (choices: {opts}): ").strip()
        if raw.isdigit() and int(raw) in allowed_day_nums:
            day_num = int(raw)
            for d in available_days:
                if d.day == day_num:
                    return d
        print("No — please enter one of:", opts)

def parse_time_10min() -> pd.Timedelta:
    """
    Accepts 'HH:MMam/pm' (case-insensitive). Allows ';' instead of ':'.
    If only 'HH' or 'HH:MM' without AM/PM → reprompt.
    Minutes are snapped to nearest 10.
    """
    while True:
        raw = input("Enter the time (e.g., 12:10PM, 12;10pm, 3:00am): ").strip()
        s = raw.replace(" ", "").replace(";", ":").upper()
        if re.fullmatch(r"\d{1,2}$", s):
            print("Please include minutes and AM/PM, e.g., 12:00PM"); continue
        if re.fullmatch(r"\d{1,2}:\d{2}$", s):
            print("Please add AM or PM, e.g., 12:10PM"); continue
        m = re.fullmatch(r"(\d{1,2}):(\d{2})(AM|PM)$", s)
        if m:
            hh = int(m.group(1)) % 12
            mm = int(m.group(2))
            mm = int(round(mm / 10.0) * 10) % 60  # snap to nearest 10-min
            if m.group(3) == "PM":
                hh += 12
            return pd.Timedelta(hours=hh, minutes=mm)
        print("Time must look like '12:10PM' (':' or ';' both OK). Try again:")

def nearest_slot(ts: pd.Timestamp, index: pd.Index) -> pd.Timestamp:
    """Snap to existing timestamp if needed (nearest)."""
    if ts in index: return ts
    i = index.get_indexer([ts], method="nearest")[0]
    return index[i]

def is_seller_by_windows(home: str, ts: pd.Timestamp) -> bool:
    """Authoritative check from seller_windows.csv: start <= ts < end."""
    day = ts.normalize().date()
    hit = sw[(sw["home"].eq(home)) & (sw["day"].eq(day)) & (sw["start"] <= ts) & (ts < sw["end"])]
    return not hit.empty

def advise_for(home: str, ts: pd.Timestamp) -> str:
    """
    Decision precedence:
    1) If a cleared trade exists at ts, report it.
    2) If ts in seller window for home → SELL (suggest top buyer from surplus).
    3) Else if home deficit → BUY (suggest top seller from surplus).
    4) Else balanced/surplus but not a window → use own/battery or grid.
    """
    # 1) honor cleared trades
    slot_trades = report[report["time"] == ts]
    if not slot_trades.empty:
        sell_rows = slot_trades[slot_trades["seller"].eq(home)]
        buy_rows  = slot_trades[slot_trades["buyer"].eq(home)]
        if not sell_rows.empty:
            total = sell_rows["quantity_kWh"].sum()
            buyers = ", ".join(sorted(set(sell_rows["buyer"])))
            return f"At {ts}, {home} is SELLING {total:.3f} kWh to {buyers}."
        if not buy_rows.empty:
            total = buy_rows["quantity_kWh"].sum()
            sellers = ", ".join(sorted(set(buy_rows["seller"])))
            return f"At {ts}, {home} is BUYING {total:.3f} kWh from {sellers}."

    # 2) authoritative availability
    if is_seller_by_windows(home, ts):
        row = surplus.loc[ts]
        buyers = (-row[row < 0]).sort_values(ascending=False)
        if not buyers.empty:
            return f"{home} should SELL now (seller window). Top buyer: {buyers.index[0]}."
        return f"{home} should SELL now (seller window). No active buyers → export to grid or wait."

    # 3) not a seller window → check if buyer
    row = surplus.loc[ts]
    if home in row.index and row[home] < 0:
        sellers = row[row > 0].sort_values(ascending=False)
        if not sellers.empty:
            return f"{home} should BUY (deficit {abs(row[home]):.2f} kW). Top seller: {sellers.index[0]}."
        return f"{home} should USE OWN/BATTERY or IMPORT from grid (no active sellers)."

    # 4) balanced or surplus but outside window
    if home in row.index and row[home] > 0:
        return f"{home} has surplus but not in a seller window → use own/battery or export to grid."
    return f"{home} is balanced (≈0 kW). Use own generation; no trade needed."


In [12]:
print("What home are you? (e.g., h1..h5)")
home = parse_home(input())

day0 = get_valid_day()
offset = parse_time_10min()

ts_raw = pd.Timestamp(day0 + offset)    # timestamps are naive/UTC-like in file
ts = nearest_slot(ts_raw, surplus.index)

print("\n--- Decision ---")
print(f"Slot selected: {ts} (nearest to your input {ts_raw})")
print(advise_for(home, ts))


What home are you? (e.g., h1..h5)
Please add AM or PM, e.g., 12:10PM

--- Decision ---
Slot selected: 2018-08-24 18:50:00+00:00 (nearest to your input 2018-08-24 18:50:00+00:00)
At 2018-08-24 18:50:00+00:00, h4 is SELLING 0.241 kWh to h2.
